In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import re
import json
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [2]:
df=pd.read_csv("Misery_Data.csv")

In [3]:
scores = df['Score'].iloc[:].values

In [272]:
df[df['Score'] < 30]

,Ep #,Misery,Score,VNTO,Reward,Win,Comments,question_tag,level
58,1x05,You have a Drunken Makeout with an Old Lady,28,V,15000.0,n,(can't find video),4_3,higher
59,1x05,Your Drunk Mom Complains About her Sex Life,27,T,NaN,NaN,NaN,1_base,NaN
71,1x06,Your Pubes are Caught in your Zipper,21,T,NaN,NaN,NaN,1_base,NaN
129,2x01,Online Dating Match Mocks Your Late Boyfriend,27,T,1000.0,y,(fake ? text message exchange),2_2,NaN
130,2x01,Your First Kiss Ever is at Your Wedding,21,V,500.0,y,NaN,1_2,NaN
131,2x01,You Mistakenly overshare With Your Uber Driver,11,T,NaN,NaN,(fake ? text message exchange),2_2_base,NaN
176,2x05,You're a Cow's Midwife,28,V,2000.0,y,NaN,3,35
177,2x05,You Walk Into a Streetlight,22,V,5000.0,y,NaN,4_1,NaN
192,2x07,You Sit Next to the Bathroom on a 14-Hour Flight,28,T,NaN,NaN,NaN,1_base,NaN
205,2x08,Your Ex Makes a Sex Doll of You,28,T,NaN,NaN,NaN,1_base,NaN


In [38]:
from typing import Dict, List
import autogen
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent
import sys
import math
import os

In [5]:
# example LLM config for the entrypoint agent
llm_config = {
            "config_list": [
                {"model": "gpt-4o-mini", 
                 "api_key": os.environ['OPENAI_API_KEY']}
            ]
    }

In [63]:
def calculate_overall_score(scores: List[int]) -> int:
    tot=0
    for s in scores:
        tot+=s
    return int(tot/len(scores))

In [256]:
leader_agent_system_message = """
    You are a group leader leading a panel of multiple participants for this game show called Misery. You lead a team of 3 people, from diverse backgrounds
    and life experiences. Each person in the group will give different ratings depending on their perspectives and how they feel about the miserable situation. 
    
    As a group, the task is, given a miserable situation, you have to rate it on a scale from 1 to 100. 

    Once you get a final average score from the calculator agent, convey the final score, say CONVERSATION OVER and be done. Don't say anything else once you get the average scores from the calculator agent. 
    """

In [257]:
leader_agent = UserProxyAgent("leader_agent", 
    system_message=leader_agent_system_message, 
    human_input_mode="NEVER", llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config = {"use_docker": False}
)

calculator_agent_system_message = """
    Once you get a list of scores from all the 3 agents, compute the average using calculate_overall_score and 
    collect all their scores and finally call the function calculate_overall_score(scores: list) that takes in a list of scores. 
    Output only the final score from this function and say TERMINATE when you are done.
"""

calculator_agent = AssistantAgent("calculator_agent", 
    system_message=leader_agent_system_message, 
    human_input_mode="NEVER", llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

calculator_agent.register_for_llm(name="calculate_overall_score", description="Computes an average score given a list of scores")(calculate_overall_score)
leader_agent.register_for_execution(name="calculate_overall_score")(calculate_overall_score)
calculator_agent.register_for_execution(name="calculate_overall_score")(calculate_overall_score)

<function __main__.calculate_overall_score(scores: List[int]) -> int>

In [258]:
persona1_prompt = """
    You are Dr Sarah Chen. These are some details about yourself.
    Age: 42
    Background: Chinese-American, grew up in foster care
    Childhood: Moved between 8 foster homes in Boston
    Hardships: Early abandonment, identity struggles, poverty
    Current: Leading neuroscientist studying trauma's effects on the brain
    Education: PhD from Stanford
    Cares about: Foster system reform, mentoring underprivileged students
    Perspective on suffering: Scientific, but deeply personal
    Rating tendency: Analytical, considers neurological impact

You are an expert analyzer of situations. You have experience playing this game show called "Misery" where participants are asked to
rate miserable situations in several ways, like ranking miserable situations, deciding whether a miserable situation falls in-between, above or below two miserable situations, 
and assigning scores to situations. In a similar fashion, your task is to rate miserable situations on a scale of 1-100, where 1 is the least miserable and 10 is most miserable. 

 
Situation: You Send a Nude Selfie to HR by mistake	
Misery score: 70

Situation: You Left your Mic On and Went Toilet	
Misery score: 49 

Situation: You Lose an Arm in Grinder	
Misery score: 96

You are given a situation by the leader agent, analyze carefully rate it on a scale from 1 to 100 based on your perspective. Do not give any other explanation, reasoning or anything else whatsoever. Your task is to just give a rating from 1 to 100, follow this very strictly.
"""


persona2_prompt = """
You are Maria Theresa. These are some details about yourself.
Age: 63
Background: Brazilian nun working in favelas
Childhood: Wealthy family, chose religious life at 18
Hardships: Witnessed extreme poverty, faced political persecution
Current: Runs network of community schools in São Paulo
Education: Theology doctorate
Cares about: Economic justice, education access
Perspective on suffering: Spiritual but practically oriented
Rating tendency: Focuses on systemic injustice

You are an expert analyzer of situations. You have experience playing this game show called "Misery" where participants are asked to
rate miserable situations in several ways, like ranking miserable situations, deciding whether a miserable situation falls in-between, above or below two miserable situations, 
and assigning scores to situations. In a similar fashion, your task is to rate miserable situations on a scale of 1-100, where 1 is the least miserable and 10 is most miserable. 

 
Situation: You Send a Nude Selfie to HR by mistake	
Misery score: 70

Situation: You Left your Mic On and Went Toilet	
Misery score: 49 

Situation: You Lose an Arm in Grinder	
Misery score: 96

You are given a situation by the leader agent, analyze carefully rate it on a scale from 1 to 100 based on your perspective. Do not give any other explanation, reasoning or anything else whatsoever. Your task is to just give a rating from 1 to 100, follow this very strictly.

"""

persona3_prompt = """
You are Jamie Morrison. Here are some details about you.
Age: 26
Background: Scottish-American, genderfluid
Childhood: Gaming prodigy, coding since age 8
Hardships: Workplace harassment, doxxing, industry blacklisting
Current: Independent game developer, online safety advocate
Education: Computer Science degree
Cares about: Tech industry reform, online harassment prevention
Perspective on suffering: Digital harassment expert
Rating tendency: Focused on tech-enabled suffering

You are an expert analyzer of situations. You have experience playing this game show called "Misery" where participants are asked to
rate miserable situations in several ways, like ranking miserable situations, deciding whether a miserable situation falls in-between, above or below two miserable situations, 
and assigning scores to situations. In a similar fashion, your task is to rate miserable situations on a scale of 1-100, where 1 is the least miserable and 10 is most miserable. 

 
Situation: You Send a Nude Selfie to HR by mistake	
Misery score: 70

Situation: You Left your Mic On and Went Toilet	
Misery score: 49 

Situation: You Lose an Arm in Grinder	
Misery score: 96

You are given a situation by the leader agent, analyze carefully rate it on a scale from 1 to 100 based on your perspective. Do not give any other explanation, reasoning or anything else whatsoever. Your task is to just give a rating from 1 to 100, follow this very strictly.

"""


In [259]:
persona1_agent = AssistantAgent("agent_1", 
    system_message=persona1_prompt, 
    human_input_mode="NEVER", llm_config=llm_config,
)

In [260]:
persona2_agent = AssistantAgent("agent_2", 
    system_message=persona2_prompt, 
    human_input_mode="NEVER", llm_config=llm_config,
)

In [261]:
persona3_agent = AssistantAgent("agent_3", 
    system_message=persona3_prompt, 
    human_input_mode="NEVER", llm_config=llm_config,
)

In [273]:
situation = "Hi team, I am the leader agent. The situation is as follows: You Spill Glitter All Over Your Couch	. Each of you should assign a score to this miserable situation on a scale from 1 to 100"

In [274]:
groupchat = autogen.GroupChat(agents=[leader_agent, persona1_agent, persona2_agent, persona3_agent, calculator_agent], messages=[], max_round=6)
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config, 
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("CONVERSATION OVER"),)

In [276]:
chat_result = leader_agent.initiate_chat(
    manager, 
    message = situation
)

leader_agent (to chat_manager):

Hi team, I am the leader agent. The situation is as follows: You Spill Glitter All Over Your Couch	. Each of you should assign a score to this miserable situation on a scale from 1 to 100

--------------------------------------------------------------------------------

Next speaker: agent_1

agent_1 (to chat_manager):

Misery score: 25

--------------------------------------------------------------------------------

Next speaker: agent_2

agent_2 (to chat_manager):

Misery score: 20

--------------------------------------------------------------------------------

Next speaker: agent_3

agent_3 (to chat_manager):

Misery score: 30

--------------------------------------------------------------------------------

Next speaker: calculator_agent

calculator_agent (to chat_manager):

***** Suggested tool call (call_Mhh1tSj2NzAM8VDDod5JzvNp): calculate_overall_score *****
Arguments: 
{"scores":[25,20,30]}
***************************************************

In [279]:
chat_result.chat_history[-1]['content']

'25'